In [6]:
class GetBERT(nn.Module):

    def __init__(self):
        super(GetBERT, self).__init__()
        self.bert_tokenizer = BertTokenizer.from_pretrained("chinese-bert-wwm-ext")
        self.bert = BertModel.from_pretrained("chinese-bert-wwm-ext")
        for param in self.bert.parameters():
            param.requires_grad = False

    def forward(self, sentence_lists):
        """
        输入句子列表(去掉了停用词的)
        """
        sentence_lists = [' '.join(x) for x in sentence_lists]
        # print('sentence_lists:'+str(sentence_lists))
        ids = self.bert_tokenizer(sentence_lists, padding=True, return_tensors="pt")
        # print('ids:'+str(ids))
        inputs = ids['input_ids']
        # print('inputs:'+str(inputs))

        embeddings = self.bert(inputs)
        # print(str(embeddings[0].shape))
        x = embeddings[0]  # 1 * 768
        # print(x.shape)
        return x


class Pre:
    def __init__(self, text):
        """
        输入一个文本
        """
        self.puncs_coarse = ['。', '!', '；', '？', '……', '\n', ' ']
        self.text = text
        self.stopwords = self.deal_wrap('dict/stop1205.txt')

    def segment(self, sentence):
        sentence_seged = jieba.cut(sentence.strip())
        outstr = ''
        for word in sentence_seged:
            if word not in self.stopwords:
                if word != '\t':
                    outstr += word
                    outstr += " "
        word_list = outstr.split(' ')
        pattern = '[A-Za-z]*[0-9]*[\'\"\%.\s\@\!\#\$\^\&\*\(\)\-\<\>\?\/\,\~\`\:\;]*[：；”“ ‘’+-——！，。？、~@#￥%……&*（）【】]*'
        t = [re.sub(pattern, "", x.strip()) for x in word_list]
        t = [x for x in t if x != '']
        return ''.join(t)

    def deal_wrap(self, filedict):
        temp = []
        for x in open(filedict, 'r', encoding='utf-8').readlines():
            temp.append(x.strip())
        return temp

    def split_sentence_coarse(self):
        """
        按照。！？“”等中文完整句子语义来分句
        1. 去除换行符、多余的空格、百分号
        2. 分句，存入列表
        :return:装着每个句子的列表（包括标点符号）
        """

        text = self.text
        sentences = []
        start = 0
        for i in range(len(text)):
            if text[i] in self.puncs_coarse:
                sentences.append(text[start:i + 1])
                start = i + 1
        if start == 0:
            sentences.append(text)
        return sentences

    def get_keywords(self, data):
        """
        如果句子太长，就进行关键词提取
        """
        from jieba import analyse
        textrank = analyse.textrank
        keywords = textrank(data, topK=8)
        return ''.join(keywords)

    def preprocess(self):
        # 分句
        sentences = self.split_sentence_coarse()
        # 对每个句子，去除里面的停用词，再连起来
        # 对每个句子，如果句子太长，长度大于20（我随便定的），就抽取八个关键词连起来
        new_sent = []
        for i in sentences:
            if len(i) < 5:
                new_sent.append(i)
                continue
            i = self.segment(i)
            if len(i) > 25:
                i = self.get_keywords(i)
            if len(i) > 20:
                i = i[:20]
            if i != '':
                new_sent.append(i)
        return new_sent


class GetData():
    def __init__(self, pos=4000, neg=3600):
        data = pd.read_csv('sentiment_classify_data/raw_comment_v2.csv')
        data = data[data['score'] != 3].reset_index()
        data['label'] = data['score'].map(lambda a: 1 if a in [4, 5] else 0)
        data.drop(['post_time','score','shop_url'],inplace=True,axis=1)
        
#         data = pd.read_excel('sentiment_classify_data/comments_raw_v1.xls')
#         data = data[data['score'] != 3].reset_index()
#         data['label'] = data['score'].map(lambda a: 1 if a in [4, 5] else 0)
#         data.drop(['id', 'score'], inplace=True, axis=1)
        
        data['content'] = [str(i) for i in list(data['content'])]
        # 原数据标签为0（负向情感）的数据有3632条，正向情感的有57262条
        data1 = data[data['label'] == 1].sample(pos)
        data0 = data[data['label'] == 0].sample(neg)
        data = pd.concat([data1, data0], axis=0, ignore_index=True)
        self.data = data

    def split_sen(self):
        x = []
        y = []
        for i in trange(len(self.data)):
            p = Pre(self.data['content'][i])
            sen_lst = p.preprocess()
            if sen_lst == []:
                continue
            x.append(sen_lst)
            y.append(self.data['label'][i])
        print(len(x))
        print(y.count(1))
        print(y.count(0))
        return x, y

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel, AutoTokenizer
from transformers import BertTokenizer, BertModel
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from copy import deepcopy as copy
import torch.optim as optim
import random
import re
import os
import jieba
from tqdm import tqdm
from tqdm import trange
def seed_torch(seed=1122):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)  # 为了禁止hash随机化，使得实验可复现
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
BERT_PATH = 'bert-base-chinese/'

In [8]:
def train_model(epoch, train_dataLoader, test_dataLoader):
    # 训练模型
    best_model = None
    train_loss = 0
    test_loss = 0
    best_loss = 100
    epoch_cnt = 0
    for _ in range(epoch):
        total_train_loss = 0
        total_train_num = 0
        total_test_loss = 0
        total_test_num = 0
        for x, y in tqdm(train_dataLoader,
                         desc='Epoch: {}| Train Loss: {}| Test Loss: {}'.format(_, train_loss, test_loss)):
        #for x, y in train_dataLoader:
            x_num = len(x)
            p = model(x,model.init_hidden())
            loss = loss_func(p, y.long())
            optimizer.zero_grad()
            loss.backward(retain_graph=True)
            optimizer.step()
            total_train_loss += loss.item()
            total_train_num += x_num
        train_loss = total_train_loss / total_train_num
        train_loss_list.append(train_loss)
        for x, y in test_dataLoader:
            x_num = len(x)
            p = model(x,model.init_hidden())
            loss = loss_func(p, y.long())
            optimizer.zero_grad()
            loss.backward(retain_graph=True)
            optimizer.step()
            total_test_loss += loss.item()
            total_test_num += x_num
        test_loss = total_test_loss / total_test_num
        test_loss_list.append(test_loss)
        
        # early stop
        if best_loss > test_loss:
            best_loss = test_loss
            best_model = copy(model)
            torch.save(best_model.state_dict(), 'bigru.pth')
            epoch_cnt = 0
        else:
            epoch_cnt += 1
            
        if epoch_cnt > early_stop:
            torch.save(best_model.state_dict(), 'bigru.pth')
            print("保存模型")
            #print(best_model.state_dict())
            break

In [9]:
def test_model(test_dataLoader_):
    pred = []
    label = []
    model_.load_state_dict(torch.load("bigru.pth"))
    model_.eval()
    total_test_loss = 0
    total_test_num = 0
    for x, y in test_dataLoader_:
        x_num = len(x)
        p = model_(x,model_.init_hidden())
        #         print('##', len(p), len(y))
        loss = loss_func(p, y.long())
        total_test_loss += loss.item()
        total_test_num += x_num
        pred.extend(p.data.squeeze(1).tolist())
        label.extend(y.tolist())
    test_loss = total_test_loss / total_test_num
    # print('##', len(pred), len(label))
    return pred, label, test_loss, test_loss_list

In [10]:
class BiGRU(torch.nn.Module):
    def __init__(self, input_dim=768,hidden_size=768, out_size=2, n_layers=1, batch_size=16):
        super(BiGRU, self).__init__()

        self.batch_size = batch_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.out_size = out_size
        self.gru = torch.nn.GRU(input_dim, hidden_size, n_layers, batch_first=True,bidirectional=True)
        self.fc1 = torch.nn.Linear(hidden_size*2, 300)
        self.fc2 = torch.nn.Linear(300, out_size)
        
    def forward(self, word_inputs, hidden):
        self.batch_size = word_inputs.shape[0]
       # hidden 就是上下文输出，output 就是 RNN 输出
        output, hidden = self.gru(word_inputs, hidden)
        # output是所有隐藏层的状态，hidden是最后一层隐藏层的状态
        output = self.fc1(output)
        output = self.fc2(output)

        # 仅仅获取 time seq 维度中的最后一个向量
        # the last of time_seq
        output = output[:,-1,:]
        output = F.log_softmax(output, dim=1)

        return output

    def init_hidden(self):
        hidden = torch.autograd.Variable(torch.zeros(2*self.n_layers, self.batch_size, self.hidden_size))
        return hidden

In [11]:
seed_torch(22)
epoch = 10
batch_size = 16
early_stop = 5
test_loss_list = []
train_loss_list = []

# 初始化模型
model = BiGRU()
model_ = BiGRU()

# 数据处理部分
gd = GetData(pos=3000, neg=3000)
x, y = gd.split_sen()
pos = y.count(1)
neg = y.count(0)
pos_train = int(pos*0.9)
neg_train = int(neg*0.9)
x1 = x[:pos]  # 3988   --- 3589train 399test
y1 = y[:pos]
x0 = x[pos:]  # 3589   ---- 3230train 359test
y0 = y[pos:]

train_x = x0[:neg_train] + x1[:pos_train]
train_y = y0[:neg_train] + y1[:pos_train]
print("训练集有"+str(len(train_x))+"个数据")

# c = list(zip(train_x, train_y))
# random.shuffle(c)
# c = random.sample(c, 50)
# train_x[:], train_y[:] = zip(*c)

test_x = x0[neg_train:] + x1[pos_train:]
test_y = y0[neg_train:] + y1[pos_train:]
print("测试集有"+str(len(test_x))+"个数据")

bert = GetBERT()
x_train = bert(train_x)
x_test = bert(test_x)
y_train = torch.tensor(train_y).float()
y_test = torch.tensor(test_y).float()
train_data = TensorDataset(x_train, y_train)
train_dataLoader = DataLoader(train_data, batch_size=batch_size,drop_last=True)
test_data = TensorDataset(x_test, y_test)
test_dataLoader = DataLoader(test_data, batch_size=batch_size,drop_last=True)

# 损失函数和优化器
loss_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)
train_model(epoch, train_dataLoader, test_dataLoader)
p, y, test_loss, test_loss_list = test_model(test_dataLoader)
ans = []
for t in p:
    if t[0]>t[1]:
        ans.append(0)
    else:
        ans.append(1)
print(accuracy_score(ans,y))

  0%|          | 0/6000 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.802 seconds.
Prefix dict has been built successfully.
100%|██████████| 6000/6000 [00:17<00:00, 348.98it/s]


5987
2995
2992
训练集有5387个数据
测试集有600个数据


Epoch: 0| Train Loss: 0| Test Loss: 0:   0%|          | 0/336 [00:00<?, ?it/s]/app/common/miniconda3/lib/python3.8/site-packages/torch/autograd/__init__.py:130: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  Variable._execution_engine.run_backward(
Epoch: 0| Train Loss: 0| Test Loss: 0: 100%|██████████| 336/336 [05:59<00:00,  1.07s/it]
Epoch: 1| Train Loss: 0.019483640937583653| Test Loss: 0.06446073078424544: 100%|██████████| 336/336 [05:51<00:00,  1.05s/it]
Epoch: 2| Train Loss: 0.02336034977079613| Test Loss: 0.06115482833135773: 100%|██████████| 336/336 [05:42<00:00,  1.02s/it]
Epoch: 3| Train Loss: 0.023567904653083088| Test Loss: 0.05827723962028284: 100%|██████████| 336/336 [05:49<00:00,  1.04s/it]
Epoch: 4| Train Loss: 0.02362254196209722| Test Loss: 0.055835924589553394: 10

0.49324324324324326
